In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]
# !{sys.executable} -m pip install pip rasterstats 

In [1]:
import ee
# ee.Authenticate()

In [2]:
ee.Initialize()

In [3]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [4]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [5]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [24]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


In [26]:
# remove cities with time out issues due to volumetry
data_volumetry_error_cities = ['MEX-Mexico_City']
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(data_volumetry_error_cities)].reset_index(drop=True)
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Monterrey,MEX-Monterrey,ADM2union,ADM2,MEX-Monterrey-ADM2,MEX,ADM2


# Extract images with population: total, with open space access and with tree access

In [11]:
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']

    # process aoi level ------
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    #read open space
    openspace_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)
    
    #load population
    pop = ee.ImageCollection('WorldPop/GP/100m/pop')
    pop = (pop.filter(ee.Filter.And(
        ee.Filter.bounds(boundary_geo_ee),
        ee.Filter.inList('year',[2020])))
        .select('population'))
    popImg = pop.mean().rename('population')

    popProj = pop.first().projection()
    popScale = popProj.nominalScale()

    # define threshold distance (m) and buffer open space areas by that to get recreation catchment
    DistanceThres = 400 # meters distance from population to be considered
    def amenityBuffer(feat):
      feat = ee.Feature(feat)
      return feat.buffer(DistanceThres)
    RecCatchment = openspace_geo_ee.map(amenityBuffer)
    RecCatchmentUnion = RecCatchment.union()
    
    #mask population by recreation catchment
    mask = ee.Image.constant(1).clipToCollection(RecCatchmentUnion).mask()
    popwOSaccess = popImg.updateMask(mask).rename('populationwOpenSpace')
    
    ## add tree cover dataset
    TML = ee.ImageCollection('projects/wri-datalab/TML').filter(ee.Filter.bounds(boundary_geo_ee))
    TreeCover = TML.reduce(ee.Reducer.mean()).rename('b1')
    
    # calcs for % population with threshold level (e.g. 10%+) of tree cover within walking distance (e.g. 400m)
    TreePctThreshold = 10 #whole numbers - 0-100, minimum percentage of tree cover threshold to consider 
    circleTheshm = ee.Kernel.circle(DistanceThres, 'meters', False)
    TreeCoverinThreshm = TreeCover.reduceNeighborhood(ee.Reducer.mean(), circleTheshm)
    popwthresTC = popImg.updateMask(TreeCoverinThreshm.gte(TreePctThreshold)).rename('populationwTreeCover')
    
    # combine images 
    combImg = popImg.addBands([popwOSaccess,popwthresTC])
    
    
    geemap.ee_export_image_to_drive(
        popImg, 
        description = boundary_id_aoi + '-WorldPop-population',
        folder='data', 
        scale=popScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 50000000000
    )
    
    geemap.ee_export_image_to_drive(
        popwOSaccess, 
        description = boundary_id_aoi + '-population-wOpenSpace',
        folder='data', 
        scale=popScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 50000000000
    )
        
    geemap.ee_export_image_to_drive(
        popwthresTC, 
        description = boundary_id_aoi + '-population-wTreeCover',
        folder='data', 
        scale=popScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 50000000000
    )

0

 geo_name: BRA-Salvador
1

 geo_name: COD-Bukavu
2

 geo_name: COD-Uvira
3

 geo_name: COG-Brazzaville
4

 geo_name: COL-Barranquilla
5

 geo_name: ETH-Addis_Ababa
6

 geo_name: ETH-Dire_Dawa
7

 geo_name: KEN-Nairobi
8

 geo_name: MDG-Antananarivo
9

 geo_name: MEX-Monterrey
10

 geo_name: RWA-Musanze


# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [12]:
# connect to s3
# aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-cities4forests' 

In [13]:
out_dir = os.getcwd()

In [27]:
# WorldPop-population
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-WorldPop-population.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/population/worldpop/v_0/'+ boundary_id + '-WorldPop-population.tif',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: BRA-Salvador-ADM4union

 boundary_id: COD-Bukavu-ADM3union

 boundary_id: COD-Uvira-ADM3union

 boundary_id: COG-Brazzaville-ADM4union

 boundary_id: COL-Barranquilla-ADM4union

 boundary_id: ETH-Addis_Ababa-ADM4union

 boundary_id: ETH-Dire_Dawa-ADM3union

 boundary_id: KEN-Nairobi-ADM3union

 boundary_id: MDG-Antananarivo-ADM4union

 boundary_id: MEX-Monterrey-ADM2union

 boundary_id: RWA-Musanze-ADM5union


In [19]:
# population-wOpenSpace
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-population-wOpenSpace.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/population/worldpop/v_0/'+ boundary_id + '-population-wOpenSpace-2020.tif',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: BRA-Salvador-ADM4union

 boundary_id: COD-Bukavu-ADM3union

 boundary_id: COD-Uvira-ADM3union

 boundary_id: COG-Brazzaville-ADM4union

 boundary_id: COL-Barranquilla-ADM4union

 boundary_id: ETH-Addis_Ababa-ADM4union

 boundary_id: ETH-Dire_Dawa-ADM3union

 boundary_id: KEN-Nairobi-ADM3union

 boundary_id: MDG-Antananarivo-ADM4union

 boundary_id: MEX-Monterrey-ADM2union

 boundary_id: RWA-Musanze-ADM5union


In [21]:
# remove cities without tree cover data availability
tml_not_available_cities = ['MEX-Monterrey']
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(tml_not_available_cities)].reset_index(drop=True)
boundary_georef

# population-wTreeCove
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-population-wTreeCover.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/population/worldpop/v_0/'+ boundary_id + '-population-wTreeCover-2020.tif',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: BRA-Salvador-ADM4union

 boundary_id: COD-Bukavu-ADM3union

 boundary_id: COD-Uvira-ADM3union

 boundary_id: COG-Brazzaville-ADM4union

 boundary_id: COL-Barranquilla-ADM4union

 boundary_id: ETH-Addis_Ababa-ADM4union

 boundary_id: ETH-Dire_Dawa-ADM3union

 boundary_id: KEN-Nairobi-ADM3union

 boundary_id: MDG-Antananarivo-ADM4union

 boundary_id: RWA-Musanze-ADM5union


In [20]:
# # all layers

# for i in range(0, len(boundary_georef)):
#     boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
#     geo_name = boundary_georef.loc[i, 'geo_name']
#     print("\n boundary_id: "+boundary_id)

#     # read local raster
#     city_file = 'data/' + boundary_id +'-WorldPop-population.tif'
#     raster_path = os.path.join(out_dir, city_file)
    
#     # upload in s3
#     s3.meta.client.upload_file(raster_path, 
#                                bucket_name, 
#                                'data/population/worldpop/v_0/'+ boundary_id + '-WorldPop-population-2020.tif',
#                                ExtraArgs={'ACL':'public-read'})

#     # read local raster
#     city_file = 'data/' + boundary_id +'-population-wTreeCover.tif'
#     raster_path = os.path.join(out_dir, city_file)
    
#     # upload in s3
#     s3.meta.client.upload_file(raster_path, 
#                                bucket_name, 
#                                'data/population/worldpop/v_0/'+ boundary_id + '-population-wTreeCover-2020.tif',
#                                ExtraArgs={'ACL':'public-read'})
    
#     # read local raster
#     city_file = 'data/' + boundary_id +'-population-wOpenSpace.tif'
#     raster_path = os.path.join(out_dir, city_file)
    
#     # upload in s3
#     s3.meta.client.upload_file(raster_path, 
#                                bucket_name, 
#                                'data/population/worldpop/v_0/'+ boundary_id + '-population-wOpenSpace-2020.tif',
#                                ExtraArgs={'ACL':'public-read'})